##### PAIR PROGRAMMING - REGRESIÓN LOGÍSTICA - DECISION TREE

Hasta ahora hemos ajustado el modelo usando una Regresión Logística, pero como hemos aprendido, podemos usar el Decision Tree en este tipo de problemas. Los objetivos de este pair programming :

In [9]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


#  Modelado y matriz de confusión
# ------------------------------------------------------------------------------

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV

#  Gestión de warnings
# ------------------------------------------------------------------------------
import warnings

In [2]:
df = pd.read_pickle("../datos/df_estan_cod.pkl")
df.head()

,satisfaction,gender,customer_type,age,type_of_travel,class,flight_distance,seat_comfort,departure/arrival_time_convenient,gate_location,...,inflight_entertainment,online_support,ease_of_online_booking,on-board_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding,departure_delay_in_minutes
0,0,0,1,1.041667,0,2,-1.400844,0,0,2,...,2,3,3,4,0,3,1,4,4,0.000000
1,0,1,1,0.291667,0,1,0.454852,0,0,4,...,3,3,3,2,1,2,3,4,4,25.833333
2,0,0,1,-1.041667,0,2,0.179747,0,0,4,...,0,3,3,4,2,2,2,2,4,0.000000
3,0,0,1,0.833333,0,2,-1.098734,0,0,4,...,2,3,3,3,0,0,2,3,2,0.000000
4,0,0,1,1.250000,0,2,-1.325738,0,0,4,...,3,2,3,3,0,0,2,3,1,0.000000


In [3]:
df.index

RangeIndex(start=0, stop=129880, step=1)

- Ajustad un modelo de Decision Tree a nuestros datos.


In [4]:
X1 = df.drop("satisfaction", axis = 1)
y1 = df["satisfaction"]

In [5]:
X1.head(2)

,gender,customer_type,age,type_of_travel,class,flight_distance,seat_comfort,departure/arrival_time_convenient,gate_location,inflight_wifi_service,inflight_entertainment,online_support,ease_of_online_booking,on-board_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding,departure_delay_in_minutes
0,0,1,1.041667,0,2,-1.400844,0,0,2,2,2,3,3,4,0,3,1,4,4,0.000000
1,1,1,0.291667,0,1,0.454852,0,0,4,0,3,3,3,2,1,2,3,4,4,25.833333


In [6]:
y1.head()

0    0
1    0
2    0
3    0
4    0
Name: satisfaction, dtype: int64

In [7]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size = 0.2, random_state = 42)

In [10]:
# creamos el objeto del modelo
arbol = DecisionTreeClassifier(random_state =0)

# ajustamos el modelo
arbol.fit(x_train1, y_train1)

DecisionTreeClassifier(random_state=0)

In [11]:
# max features para obtener la profudidad máxima. En nuestro caso parece que está cercana al 5

max_features = np.sqrt(len(x_train1.columns))
max_features

4.47213595499958

In [12]:
# max depth

print(arbol.tree_.max_depth)

34


In [13]:
# hacemos las predicciones sobre los dos set de datos el X_test y el X_train

y_pred_test_esta = arbol.predict(x_test1)
y_pred_train_esta = arbol.predict(x_train1)

In [14]:
def metricas(clases_reales_test, clases_predichas_test, clases_reales_train, clases_predichas_train, modelo):
    
    # para el test
    accuracy_test = accuracy_score(clases_reales_test, clases_predichas_test)
    precision_test = precision_score(clases_reales_test, clases_predichas_test)
    recall_test = recall_score(clases_reales_test, clases_predichas_test)
    f1_test = f1_score(clases_reales_test, clases_predichas_test)
    kappa_test = cohen_kappa_score(clases_reales_test, clases_predichas_test)

    # para el train
    accuracy_train = accuracy_score(clases_reales_train, clases_predichas_train)
    precision_train = precision_score(clases_reales_train, clases_predichas_train)
    recall_train = recall_score(clases_reales_train, clases_predichas_train)
    f1_train = f1_score(clases_reales_train, clases_predichas_train)
    kappa_train = cohen_kappa_score(clases_reales_train, clases_predichas_train)
    

    
    df = pd.DataFrame({"accuracy": [accuracy_test, accuracy_train], 
                       "precision": [precision_test, precision_train],
                       "recall": [recall_test, recall_train], 
                       "f1": [f1_test, f1_train],
                       "kapppa": [kappa_test, kappa_train],
                       "set": ["test", "train"]})
    
    df["modelo"] = modelo
    return df

In [15]:
# sacamos las métricas para ver si hay overfitting o underfitting, para modificar la profundidad en función de estos resultados

dt_resultados = metricas(y_test1, y_pred_test_esta, y_train1, y_pred_train_esta, "Decission Tree I")
dt_resultados

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.931976,0.925236,0.923255,0.924244,0.86252,test,Decission Tree I
1,1.000000,1.000000,1.000000,1.000000,1.00000,train,Decission Tree I


COMENTARIO:

- Vemos un claro overfitting porque los valores del train son mucho mejores que los del test. 
- El modelo se ha aprendido los datos de memoria; por tanto, habrá que reducir la profundidad del modelo.

In [16]:
# lo primero que tenemos que hacer es definir un diccionario con los hiperparámetros que queremos modificar y los valores que queremos 

param = {"max_depth": [7,15, 34], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [1,2,3,4],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]}

In [17]:
# una vez creado el diccionario iniciaremos el modelo con GridSearch

gs = GridSearchCV(
            estimator=DecisionTreeClassifier(random_state= 42), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1) # para que no nos printee ningún mensaje en pantalla

In [18]:
# ajustamos el modelo que acabamos de definir en el GridSearch

gs.fit(x_train1, y_train1)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [7, 15, 34], 'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             verbose=-1)

In [19]:
# usamos el método best_estimator_ para obtener el mejor ajuste posible
 
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeClassifier(max_depth=15, max_features=4, min_samples_leaf=10,
                       min_samples_split=10, random_state=42)

In [20]:
param1 = {"max_depth": [7,10,15], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [1,2,3,4],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [5, 10, 15],
        "min_samples_leaf": [5, 10, 15]}

In [21]:
gs1 = GridSearchCV(
            estimator=DecisionTreeClassifier(random_state= 42), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1) # para que no nos printee ningún mensaje en pantalla

In [22]:
gs1.fit(x_train1, y_train1)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [7, 15, 34], 'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             verbose=-1)

In [26]:
mejor_modelo2 = gs1.best_estimator_
mejor_modelo2

DecisionTreeClassifier(max_depth=15, max_features=4, min_samples_leaf=10,
                       min_samples_split=10, random_state=42)

In [27]:
y_pred_test2 = mejor_modelo.predict(x_test1)
y_pred_train2 = mejor_modelo.predict(x_train1)

- Calculad las métricas de nuestro nuevo modelo.


In [28]:
#sacamos otra vez los resultados de este segundo entrenamiento con ajuste de profundidad

df_resultados1 = metricas(y_test1, y_pred_test2, y_train1, y_pred_train2, "Decission Tree II")
df_resultados1

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.928665,0.917815,0.924026,0.92091,0.855946,test,Decission Tree II
1,0.938809,0.927828,0.938028,0.93290,0.876664,train,Decission Tree II


COMENTARIO:
- Hemos conseguido reducir el overfitting del train, cuyos valores eran todos 1, y ahora los valores están balanceados.

- Comparad las métricas con el modelo hecho hasta ahora. ¿Cuál es mejor?

In [32]:
df_logis = pd.read_pickle('../datos/resultados_log.pkl')
df_logis.head()

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.879427,0.871209,0.858672,0.864895,0.756042,test,Regresión Logística
1,0.878157,0.868882,0.861284,0.865067,0.754002,train,Regresión Logística


In [41]:
# concatenamos los 2 dataframes para poder comparar bien las métricas y analizarlas

df_metricas = pd.concat([dt_resultados, df_resultados1, df_logis], axis = 0)

In [42]:
df_metricas

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.931976,0.925236,0.923255,0.924244,0.862520,test,Decission Tree I
1,1.000000,1.000000,1.000000,1.000000,1.000000,train,Decission Tree I
0,0.928665,0.917815,0.924026,0.920910,0.855946,test,Decission Tree II
1,0.938809,0.927828,0.938028,0.932900,0.876664,train,Decission Tree II
0,0.879427,0.871209,0.858672,0.864895,0.756042,test,Regresión Logística
1,0.878157,0.868882,0.861284,0.865067,0.754002,train,Regresión Logística


COMENTARIO:
...

In [44]:
df_metricas.to_pickle('../datos/comparacion_resultados_metricas.pkl')